In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import pipeline

import csv
import pandas as pd
pd.options.display.max_rows = 100

import pickle
import math
import numpy as np

from collections import Counter
from tqdm import tqdm

from konlpy.tag import Kkma
from konlpy.utils import pprint
kkma = Kkma()

In [2]:
#https://huggingface.co/kykim/bert-kor-base
#model.eval() for model information

tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
model = AutoModelForMaskedLM.from_pretrained("kykim/bert-kor-base")

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
unmask = pipeline(task="fill-mask", model=model, tokenizer=tokenizer, top_k=1)

In [4]:
def main(filename):
    data_path = "{FILE}.csv".format(FILE=filename)
    vocab_dict = pd.read_pickle('vocab_pos_df.pickle').set_index('token').to_dict()['pos']
    inputs = pd.read_csv(data_path)
    
    output = []
    
    with tqdm(total=inputs.shape[0]) as pbar:
        for index, row in inputs.iterrows():
            
            sentence = row['Item']
            responses = unmask(sentence)
            
            for d in responses:
                for item in range(len(responses)):                
                    for v in vocab_dict:
                        if responses[item]['token_str'] == v:
                            responses[item]['pos'] = vocab_dict[v]
                            
            output.append(responses)
            
            pbar.update(1)
    
    pickle.dump(output, open("{TASK}_output.csv".format(TASK=filename), 'wb'))
    
    return output

In [5]:
output = main("3cond_bert")

100%|█████████████████████████████████████████| 360/360 [00:48<00:00,  7.42it/s]


In [6]:
len(output)

360